In [1]:
import random

class Order:
    def __init__(self, trader_id, otype, price, qty, time):
        self.trader_id = trader_id
        self.otype = otype  
        self.price = price
        self.qty = qty
        self.time = time

class Trader:
    def __init__(self, tid):
        self.tid = tid

    def getorder(self, time, time_left, lob):
        raise NotImplementedError("This method should be overridden by subclasses")

class Trader_ZIP(Trader):
    def __init__(self, tid):
        super().__init__(tid)

    def getorder(self, time, time_left, lob):
       
        otype = random.choice(['Bid', 'Ask'])
        price = random.randint(1, 500)
        qty = random.randint(1, 10)
        return Order(self.tid, otype, price, qty, time)

class LimitOrderBook:
    def __init__(self):
        self.bid_lob = {'best': None, 'worst': 1, 'n': 0, 'lob': []}
        self.ask_lob = {'best': None, 'worst': 500, 'n': 0, 'lob': []}
        self.tape = []

    def add_order(self, order):
        if order.otype == 'Bid':
            self.bid_lob['lob'].append(order)
            self.bid_lob['lob'].sort(key=lambda x: x.price, reverse=True)
            self.bid_lob['n'] += 1
            self.bid_lob['best'] = self.bid_lob['lob'][0].price
        elif order.otype == 'Ask':
            self.ask_lob['lob'].append(order)
            self.ask_lob['lob'].sort(key=lambda x: x.price)
            self.ask_lob['n'] += 1
            self.ask_lob['best'] = self.ask_lob['lob'][0].price

        self.match_orders()

    def match_orders(self):
        while self.bid_lob['best'] is not None and self.ask_lob['best'] is not None and self.bid_lob['best'] >= self.ask_lob['best']:
            bid_order = self.bid_lob['lob'][0]
            ask_order = self.ask_lob['lob'][0]

            trade_qty = min(bid_order.qty, ask_order.qty)
            trade_price = ask_order.price
            trade_time = max(bid_order.time, ask_order.time)

            self.tape.append({'price': trade_price, 'qty': trade_qty, 'time': trade_time})

            bid_order.qty -= trade_qty
            ask_order.qty -= trade_qty

            if bid_order.qty == 0:
                self.bid_lob['lob'].pop(0)
                self.bid_lob['n'] -= 1
                self.bid_lob['best'] = self.bid_lob['lob'][0].price if self.bid_lob['n'] > 0 else None

            if ask_order.qty == 0:
                self.ask_lob['lob'].pop(0)
                self.ask_lob['n'] -= 1
                self.ask_lob['best'] = self.ask_lob['lob'][0].price if self.ask_lob['n'] > 0 else None

    def print_lob(self):
        print("BID LOB:")
        for order in self.bid_lob['lob']:
            print(f"Trader: {order.trader_id}, Price: {order.price}, Qty: {order.qty}, Time: {order.time}")
        print("ASK LOB:")
        for order in self.ask_lob['lob']:
            print(f"Trader: {order.trader_id}, Price: {order.price}, Qty: {order.qty}, Time: {order.time}")

    def print_tape(self):
        print("Tape:")
        for trade in self.tape:
            print(f"Price: {trade['price']}, Qty: {trade['qty']}, Time: {trade['time']}")

def market_session(sess_id, starttime, endtime, trader_spec, order_schedule, dump_flags, verbose):
    
    lob = LimitOrderBook()
    traders = {tid: Trader_ZIP(tid) for tid in trader_spec}

    time = starttime
    while time < endtime:
        time_left = endtime - time

        
        tid = random.choice(list(traders.keys()))
        order = traders[tid].getorder(time, time_left, lob)
        
        if verbose:
            print(f"Trader {tid} generated order: {order.otype} {order.qty} @ {order.price}")

        
        lob.add_order(order)

        time += random.uniform(0.1, 1.0)  

    if dump_flags['dump_lobs']:
        lob.print_lob()
    if dump_flags['dump_tape']:
        lob.print_tape()


trial_id = 1
start_time = 0.0
end_time = 100.0
traders_spec = ['Trader1', 'Trader2', 'Trader3', 'Trader4']
order_sched = {}  
dump_flags = {'dump_lobs': True, 'dump_tape': True}
verbose = True

market_session(trial_id, start_time, end_time, traders_spec, order_sched, dump_flags, verbose)

Trader Trader3 generated order: Ask 9 @ 74
Trader Trader3 generated order: Bid 10 @ 350
Trader Trader4 generated order: Bid 10 @ 316
Trader Trader3 generated order: Ask 5 @ 296
Trader Trader2 generated order: Ask 10 @ 421
Trader Trader4 generated order: Bid 4 @ 85
Trader Trader4 generated order: Ask 2 @ 497
Trader Trader4 generated order: Bid 6 @ 261
Trader Trader2 generated order: Ask 7 @ 111
Trader Trader3 generated order: Bid 5 @ 308
Trader Trader4 generated order: Ask 8 @ 447
Trader Trader4 generated order: Ask 3 @ 55
Trader Trader1 generated order: Bid 10 @ 135
Trader Trader4 generated order: Bid 4 @ 13
Trader Trader3 generated order: Bid 4 @ 196
Trader Trader1 generated order: Ask 10 @ 240
Trader Trader3 generated order: Ask 1 @ 443
Trader Trader1 generated order: Ask 2 @ 52
Trader Trader2 generated order: Bid 10 @ 407
Trader Trader1 generated order: Bid 2 @ 445
Trader Trader3 generated order: Bid 6 @ 434
Trader Trader2 generated order: Bid 7 @ 38
Trader Trader2 generated order: 

# Explanation

The code represents a simplified market simulation where traders generate buy (Bid) and sell (Ask) orders, which are processed by a Limit Order Book (LOB). The key components include the Order class, representing market orders with attributes like trader ID, order type, price, quantity, and time of creation, and the Trader class, serving as the base for different trader types, with the getorder method designed for specific trading strategies. The Trader_ZIP subclass of Trader is meant to implement the ZIP (Zero-Intelligence Plus) trading strategy but currently generates random orders instead. The LimitOrderBook class manages buy and sell orders, maintaining lists for bids and asks, processing orders to match and execute trades, and recording completed trades in the tape. The market_session function sets up and runs a simulated trading session by initializing the LOB and traders, generating random orders, processing them, and advancing the simulation time. The dump_flags control whether the LOB and trade tape are printed at the end of the session, while the verbose flag controls the printing of order details during the session. The code executes a single market session with four traders generating and processing random orders through the LOB.

# Output

The simulation output provides insights into the market's behavior during the session, starting with a list of generated orders that specify the trader ID, order type (Bid or Ask), quantity, and price. For example, an order like "Trader3 generated order: Ask 5 @ 288" indicates that Trader3 placed an Ask order to sell 5 units at a price of 288. The Limit Order Book (LOB) output reveals the state of active buy and sell orders at the session's end, with Bid orders sorted by descending price and Ask orders by ascending price. For instance, the highest Bid from Trader1 is for 1 unit at 118, while the lowest Ask from Trader2 is for 3 units at 167. The tape logs executed trades, such as one at a price of 288 for 5 units early in the session. The randomness in order generation reflects the simple random strategy used by the traders, resulting in a variety of bids and asks with significant gaps that can cause market inefficiencies. Although the LOB effectively manages order matching, not all orders are immediately matched due to random pricing, leaving a mix of unmatched bids and asks at the session's end. The market behavior observed demonstrates inefficiency due to the non-strategic, random actions of the traders, offering a baseline understanding of how more sophisticated traders, like a potential ZIPSH trader, might need to adapt to succeed in this noisy environment. This output sets the stage for further experiments, such as introducing adaptive strategies to enhance market efficiency.

In [2]:
import random
import math



class Trader_ZIP:
    def __init__(self, tid, balance):
        self.tid = tid
        self.balance = balance
        self.offer_price = None
        self.limit_price = random.randint(50, 250)  
        self.job = random.choice(['Bid', 'Ask'])  
        self.active = False
        self.ga_params = {'mutation_rate': 0.1, 'crossover_rate': 0.7}  # GA params
        self.genome = {'margin': random.uniform(-0.05, 0.05)}
    
    def getorder(self, time, time_left, lob):
        print(f"Trader {self.tid}: Job = {self.job}, Best Bid = {lob['best_bid']}, Best Ask = {lob['best_ask']}, Limit Price = {self.limit_price}")
        
        if self.job == 'Bid':
            self.offer_price = lob['best_bid'] + 1
            if self.offer_price > self.limit_price:
                self.offer_price = self.limit_price
        elif self.job == 'Ask':
            self.offer_price = lob['best_ask'] - 1
            if self.offer_price < self.limit_price:
                self.offer_price = self.limit_price

        print(f"Trader {self.tid} generated order: Price = {self.offer_price}, Qty = 1")
        return {'tid': self.tid, 'price': self.offer_price, 'qty': 1}

    def adapt(self, success):
        if success:
            self.genome['margin'] += self.genome['margin'] * 0.1
        else:
            self.genome['margin'] -= self.genome['margin'] * 0.1
        
  
        self.genome['margin'] = min(max(self.genome['margin'], -0.05), 0.05)

    def genetic_algorithm(self, population):
        
        offspring = []
        for i in range(len(population) // 2):
            parent1 = population[random.randint(0, len(population) - 1)]
            parent2 = population[random.randint(0, len(population) - 1)]
            child = {'margin': (parent1['margin'] + parent2['margin']) / 2}
            if random.random() < self.ga_params['mutation_rate']:
                child['margin'] += random.uniform(-0.01, 0.01)
            offspring.append(child)
        return offspring



def market_session(sess_id, start_time, end_time, traders_spec, order_schedule, dump_flags, verbose):
    time = start_time
    tape = []
    traders = {}
    for s in traders_spec['sellers']:
        for i in range(s[1]):
            traders['S' + str(i)] = Trader_ZIP('S' + str(i), balance=0)

    for b in traders_spec['buyers']:
        for i in range(b[1]):
            traders['B' + str(i)] = Trader_ZIP('B' + str(i), balance=0)

    while time < end_time:
        time += random.uniform(0.01, 1.0)
        if time > end_time:
            break
        
        for tid in traders:
            trader = traders[tid]
            lob = {'best_bid': random.randint(50, 150), 'best_ask': random.randint(150, 250)}
            order = trader.getorder(time, end_time - time, lob)
            if order:
                tape.append({'time': time, 'tid': tid, 'price': order['price'], 'qty': order['qty']})
                trader.adapt(success=True)  

    if dump_flags and dump_flags.get('dump_tape', False):
        for trade in tape:
            print(f"Price: {trade['price']}, Qty: {trade['qty']}, Time: {trade['time']}")
    
    return tape



supply_schedule = [{'from': 0, 'to': 100, 'ranges': [(50, 100)]}]
demand_schedule = [{'from': 0, 'to': 100, 'ranges': [(150, 200)]}]

order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': 30, 'timemode': 'drip-poisson'}



traders_spec = {'sellers': [('ZIP', 10)], 'buyers': [('ZIC', 5)]}



dump_flags = {'dump_tape': True}

market_session(sess_id='test', start_time=0, end_time=100,
               traders_spec=traders_spec, order_schedule=order_sched,
               dump_flags=dump_flags, verbose=True)

Trader S0: Job = Ask, Best Bid = 80, Best Ask = 169, Limit Price = 147
Trader S0 generated order: Price = 168, Qty = 1
Trader S1: Job = Ask, Best Bid = 141, Best Ask = 164, Limit Price = 68
Trader S1 generated order: Price = 163, Qty = 1
Trader S2: Job = Bid, Best Bid = 118, Best Ask = 186, Limit Price = 231
Trader S2 generated order: Price = 119, Qty = 1
Trader S3: Job = Bid, Best Bid = 52, Best Ask = 196, Limit Price = 80
Trader S3 generated order: Price = 53, Qty = 1
Trader S4: Job = Ask, Best Bid = 78, Best Ask = 225, Limit Price = 78
Trader S4 generated order: Price = 224, Qty = 1
Trader S5: Job = Ask, Best Bid = 126, Best Ask = 156, Limit Price = 68
Trader S5 generated order: Price = 155, Qty = 1
Trader S6: Job = Ask, Best Bid = 71, Best Ask = 210, Limit Price = 119
Trader S6 generated order: Price = 209, Qty = 1
Trader S7: Job = Bid, Best Bid = 110, Best Ask = 247, Limit Price = 65
Trader S7 generated order: Price = 65, Qty = 1
Trader S8: Job = Bid, Best Bid = 146, Best Ask = 17

[{'time': 0.2340727946190835, 'tid': 'S0', 'price': 168, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'S1', 'price': 163, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'S2', 'price': 119, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'S3', 'price': 53, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'S4', 'price': 224, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'S5', 'price': 155, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'S6', 'price': 209, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'S7', 'price': 65, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'S8', 'price': 147, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'S9', 'price': 213, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'B0', 'price': 138, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'B1', 'price': 206, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'B2', 'price': 65, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'B3', 'price': 51, 'qty': 1},
 {'time': 0.2340727946190835, 'tid': 'B4', 'price': 

# Explanation

The code simulates a basic market environment where traders, represented by the Trader_ZIP class, generate buy or sell orders based on their roles (buyer or seller) and market conditions. Each Trader_ZIP instance is initialized with a unique ID, balance, a randomly assigned limit price (the maximum or minimum price they are willing to buy or sell at), and either a 'Bid' or 'Ask' role. The class also includes genetic algorithm (GA) parameters and a margin value that influences pricing strategies. During the market session, traders generate orders based on the best bid and ask prices from the Limit Order Book (LOB), ensuring their offers are competitive yet within their limit prices. After a trade, the trader adapts their strategy by adjusting the margin, becoming more aggressive if successful or more conservative if not, while the GA helps evolve strategies over time by combining and mutating parent strategies. The market session, defined by start and end times, runs in a loop where traders generate orders at each time step based on current LOB conditions, and the trades are recorded in a tape. If enabled, the tape of trades is printed at the session's end. The supply and demand schedules define price ranges for market activity, though they aren't deeply integrated into this simulation. The output logs all trades, providing insight into how traders interact under various conditions and demonstrating the effectiveness of the simple GA-based adaptation mechanism.

# Output

The output log of orders generated by traders in the market simulation highlights the decisions made by each trader throughout the simulation, providing insights into their behavior and the market's functioning. Each trader is randomly assigned a role as either a buyer (Bid) or seller (Ask), determining whether they will generate a bid or ask order. The system logs the current best bid and ask prices from the Limit Order Book (LOB), each trader's limit price (the maximum price a buyer will pay or the minimum a seller will accept), the generated order's price, and the order quantity, which is consistently set to 1 for simplicity. For buyers, the price is set just above the current best bid without exceeding the limit price, while for sellers, it is set just below the best ask, staying above the limit price. Traders may adapt their strategies after generating orders, with successful trades potentially leading to more aggressive or conservative future bids or asks. The diversity in bid and ask prices across traders reflects the random initialization of limit prices and varying LOB conditions. The interaction between the best bid and ask prices and traders' decisions creates dynamic price fluctuations, with some orders closely matching LOB prices while others diverge due to differing limit prices. This output shows that the market simulation functions as intended, with traders making decisions influenced by their roles, the LOB, and their limit prices. The traders actively engage in the market, generating valid prices and demonstrating an adaptive behavior that could lead to improved strategies. Further exploration could involve increasing the complexity of the adaptation mechanism, such as more rigorous application of the genetic algorithm, and analyzing the impact of different trader proportions on market dynamics.